In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
data_root = "../../../../../../Desktop/DevLongEvalTrainCollection/"
! tree {data_root} -L 3

../../../../../../Desktop/DevLongEvalTrainCollection/
├── 2023_01
│   ├── English
│   │   ├── Documents
│   │   ├── Qrels
│   │   └── Queries
│   ├── French
│   │   ├── Documents
│   │   ├── Qrels
│   │   └── Queries
│   └── count_docs.sh
└── _SUCCESS

10 directories, 2 files


In [3]:
import os
os.environ['SPARK_LOCAL_DIR'] = ''
print(os.environ.get('SPARK_LOCAL_DIR'))

In [11]:
from longeval.spark import get_spark
import os

os.environ['SPARK_LOCAL_DIR'] = '/Users/ikafrulbasha/Library/Python/3.9/lib/python/site-packages/pyspark'
# print(os.environ['SPARK_LOCAL_DIR'])

spark = get_spark()
spark

25/01/24 17:42:13 WARN Utils: Your hostname, Imrans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.106 instead (on interface en0)
25/01/24 17:42:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/24 17:42:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/01/24 17:42:14 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 57986)
Traceback (most recent call last):
  File "/Users/ikafrulbasha/.pyenv/versions/3.9.16/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/ikafrulbasha/.pyenv/versions/3.9.16/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/ikafrulbasha/.pyenv/versions/3.9.16/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/ikafrulbasha/.pyenv/versions/3.9.16/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/ikafrulbasha/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Users/ikafrulbasha/.pyenv/versions/3.9.16/lib/python3.9/site-packag

In [18]:
from longeval.collection import RawCollection

train_eng = RawCollection(
    spark, f"{data_root}/2023_01/English"
)
docs = train_eng.documents.cache()
print("Number of documents: " + str(docs.count()))

Number of documents: 1260


25/01/24 17:48:47 WARN CacheManager: Asked to cache already cached data.        


In [13]:
docs.show(5, vertical=True, truncate=80)

-RECORD 0------------------------------------------------------------------------------------
 contents | Forget the Nukes:\nTunnels\nAre North\nKorea's Secret Weapon\n|\nThe\nNationa... 
 docid    | doc012310000001                                                                  
-RECORD 1------------------------------------------------------------------------------------
 contents | Project Perfect Mod ::\nView topic\n- Who will take over the island?\nDo you ... 
 docid    | doc012310000002                                                                  
-RECORD 2------------------------------------------------------------------------------------
 contents | "Lawyer James B. Donovan, widely known for negotiating the release of...\nNie... 
 docid    | doc012310000003                                                                  
-RECORD 3------------------------------------------------------------------------------------
 contents | What Rex Tillerson'\ns Nomination Means for Russ

In [14]:
train_eng.queries.show(5, vertical=True)

-RECORD 0---------------------
 qid   | q012318              
 query | case over the border 
-RECORD 1---------------------
 qid   | q012396              
 query | water atlantic       
-RECORD 2---------------------
 qid   | q0123180             
 query | blanquette de vea... 
-RECORD 3---------------------
 qid   | q0123240             
 query | gift woman           
-RECORD 4---------------------
 qid   | q0123387             
 query | Government           
only showing top 5 rows



In [8]:
train_eng.qrels.show(5, vertical=True)

-RECORD 0----------------
 qid   | q012318         
 rank  | 0               
 docid | doc012303114898 
 rel   | 0               
-RECORD 1----------------
 qid   | q012318         
 rank  | 0               
 docid | doc012307806130 
 rel   | 1               
-RECORD 2----------------
 qid   | q012318         
 rank  | 0               
 docid | doc012311314092 
 rel   | 0               
-RECORD 3----------------
 qid   | q012318         
 rank  | 0               
 docid | doc012301310209 
 rel   | 0               
-RECORD 4----------------
 qid   | q012318         
 rank  | 0               
 docid | doc012311608989 
 rel   | 0               
only showing top 5 rows



In [39]:
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts="http://localhost:9200",
)
client.info()

INFO:opensearch:GET http://localhost:9200/ [status:200 request:0.018s]


{'name': 'ec8647ef7bdf',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'ZfUnTr8ZT-Ov2PK_CobhPw',
 'version': {'distribution': 'opensearch',
  'number': '2.18.0',
  'build_type': 'tar',
  'build_hash': '99a9a81da366173b0c2b963b26ea92e15ef34547',
  'build_date': '2024-10-31T19:08:04.231254959Z',
  'build_snapshot': False,
  'lucene_version': '9.12.0',
  'minimum_wire_compatibility_version': '7.10.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

In [13]:
# add a default index template that sets replicas to 0
# also make an indexable contents and docid fields
client.indices.put_template(
    name="default",
    body={
        "index_patterns": ["*"],
        "settings": {
            "number_of_replicas": 0,
            "number_of_shards": 4,
        },
        "mappings": {
            "properties": {
                "contents": {
                    "type": "text",
                },
                "docid": {
                    "type": "keyword",
                },
            },
        },
    },
)

{'acknowledged': True}

In [14]:
# list all indices
print(client.cat.indices())

green open .opensearch-observability NjP0f8pEQRWGF5Dzevrkbw 1 0 0 0  208b  208b
green open .plugins-ml-config        p4PGo4QqTui16JhBHpaqvQ 1 0 1 0   4kb   4kb
green open .ql-datasources           t64TwkMFS9WIVqms2plQvA 1 0 0 0  208b  208b
green open .kibana_1                 FP__coYdQUCWeAtADNY-iA 1 0 1 0 5.3kb 5.3kb



In [ ]:
from longeval.collection import ParquetCollection
from longeval.spark import get_spark

spark = get_spark()

root = "../../data/longeval/parquet/"
collection = ParquetCollection(spark, f"{root}/2023_01/Documents/Json")
collection.documents.show(n=5)

In [20]:
index_name = "test_index_1"  

document = {
    "contents": "This is a test document.",
    "docid": "12345"
}

response = client.index(
    index=index_name,
    body=document,
    id="1"  
)

In [21]:
response = client.indices.get_mapping(index=index_name)
print(response)

{'test_index_1': {'mappings': {'properties': {'contents': {'type': 'text'}, 'docid': {'type': 'keyword'}}}}}


In [22]:
index_name = "test_index_1"  

document = {
    "contents": "This is the 2nd test document.",
    "docid": "111"
}

response = client.index(
    index=index_name,
    body=document,
    id="2"  
)

In [23]:
index_name = "animal_index"  

doc_1 = {
    "contents": "I am a dog. I like the color green.",
    "docid": "111"
}

doc_2 = {
    "contents": "I am a cat. I like the color purple but sometimes red.",
    "docid": "222"
}

doc_3 = {
    "contents": "I am a horse. I dont like the color orange but I like blue.",
    "docid": "333"
}

response = client.index(
    index=index_name,
    body=doc_1,
    id="1"  
)

response = client.index(
    index=index_name,
    body=doc_2,
    id="2"  
)

response = client.index(
    index=index_name,
    body=doc_3,
    id="3"  
)

In [21]:
from opensearchpy import OpenSearch

query = {
    "query": {
        "match": {
            "contents": "like" # appears twice in doc 3
        }
    }
}

response = client.search(
    index="animal_index",  
    body=query             
)

print(response) # max score matches the horse statement

{'took': 5, 'timed_out': False, '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 0.17382786, 'hits': [{'_index': 'animal_index', '_id': '3', '_score': 0.17382786, '_source': {'contents': 'I am a horse. I dont like the color orange but I like blue.', 'docid': '333'}}, {'_index': 'animal_index', '_id': '1', '_score': 0.1473054, '_source': {'contents': 'I am a dog. I like the color green.', 'docid': '111'}}, {'_index': 'animal_index', '_id': '2', '_score': 0.13198866, '_source': {'contents': 'I am a cat. I like the color purple but sometimes red.', 'docid': '222'}}]}}


In [23]:
from opensearchpy import OpenSearch

response = client.count(index="animal_index")
total_documents = response['count']
print(f"Total number of documents in the index: {total_documents}")

Total number of documents in the index: 3


In [46]:
test_index = "test_index"  

doc_1 = {
    "contents": "Sample 1",
    "docid": "111"
}

doc_2 = {
    "contents": "Sample 2",
    "docid": "222"
}

doc_3 = {
    "contents": "Sample 3",
    "docid": "333"
}

response = client.index(
    index=test_index,
    body=doc_1,
    id="1"  
)

response = client.index(
    index=test_index,
    body=doc_2,
    id="2"  
)

response = client.index(
    index=test_index,
    body=doc_3,
    id="3"  
)

response = client.count(index=test_index)
total_documents = response['count']
print(f"Total number of documents in the index: {total_documents}")

response = client.indices.delete(index=test_index)
print(f"Index '{test_index}' deleted successfully!")



INFO:opensearch:PUT http://localhost:9200/test_index/_doc/1 [status:201 request:0.197s]
INFO:opensearch:PUT http://localhost:9200/test_index/_doc/2 [status:201 request:0.003s]
INFO:opensearch:PUT http://localhost:9200/test_index/_doc/3 [status:201 request:0.002s]
INFO:opensearch:POST http://localhost:9200/test_index/_count [status:200 request:0.004s]
INFO:opensearch:DELETE http://localhost:9200/test_index [status:200 request:0.048s]


Total number of documents in the index: 0
Index 'test_index' deleted successfully!


In [62]:
from opensearchpy import OpenSearch, helpers
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

long_eval_dev_index = "long_eval_dev_docs"

def get_opensearch_client():
    """Returns an OpenSearch client instance."""
    return OpenSearch(
        hosts=[{"host": "localhost", "port": 9200}],
        http_compress=True
    )

def index_doc_into_opensearch(row):
    doc = row.asDict()
    doc_id = doc.get('docid')
    contents = doc.get('contents', "")
    if not doc_id or not contents:  
        return None

    return {
        "_index": long_eval_dev_index,
        "_id": doc_id,  
        "_source": {
            "contents": contents
        }
    }

batch_size = 100  # Dev has 1260 docs, so can tune this later
docs_rdd = docs.rdd.map(index_doc_into_opensearch)

def index_batch(partition):
    """
    Indexes a batch of documents into OpenSearch.
    """
    try:
        client = get_opensearch_client()
        actions = list(partition)
        actions = [{"_index": index_name, "_source": record} for record in partition if isinstance(record, dict)]
        if actions:  
            helpers.bulk(client, actions)
    except Exception as partition_error:
        logger.error(f"Error indexing partition: {partition_error}", exc_info=True)

docs.foreachPartition(index_batch)

client = get_opensearch_client()
response = client.count(index=long_eval_dev_index)
total_documents = response['count']
print(f"Total number of LongEval dev documents in the index: {total_documents}")

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [long_eval_dev_docs]', long_eval_dev_docs, index_or_alias)

25/01/25 00:32:13 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1983484 ms exceeds timeout 120000 ms
25/01/25 00:32:13 WARN SparkContext: Killing executors is not supported by current scheduler.
25/01/25 00:32:16 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [60]:
response = client.count(index=long_eval_dev_index)
total_documents = response['count']
print(f"Total number of LongEval dev documents in the index: {total_documents}")

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [long_eval_dev_docs]', long_eval_dev_docs, index_or_alias)